In [8]:
import json
import os
from glob import glob
from random import sample
import librosa
from shutil import copyfile, rmtree
from distutils.dir_util import copy_tree
import torch
import torch.nn as nn
from torch.autograd import Variable
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers, all_characters):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.all_characters = all_characters
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
    
    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = self.all_characters.index(string[c])
        return Variable(tensor)


def evaluate(model, prime_str=['1107n9n9n9n9n9n9n9n9n9n9'], predict_len=1000, temperature=0.8):
    hidden = model.init_hidden()
    prime_input = model.char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = model.all_characters[top_i]
        predicted.append(predicted_char)
        inp = model.char_tensor([predicted_char])

    return predicted

In [10]:
with open('all_characters.json', 'r') as f:
    all_characters = json.loads(f.read())

with open('params.json', 'r') as f:
    params = json.loads(f.read())

n_characters = len(all_characters)
hidden_size = params['hidden_size']
n_layers = params['n_layers']

model = RNN(n_characters, hidden_size, n_characters, n_layers, all_characters)
model.load_state_dict(torch.load('model.pth'))
model.eval()

RNN(
  (encoder): Embedding(15, 100)
  (gru): GRU(100, 100)
  (decoder): Linear(in_features=100, out_features=15, bias=True)
)

In [15]:
temperature = 0.8
levels = ['Expert', 'ExpertPlus']
levels = ['Expert']
song_ids = ['vodka']

In [14]:
for song_id in song_ids:
    with open('selected_songs/' + song_id + '/info.dat', 'r') as f:
        std_info = json.loads(f.read())

    std_info['_levelAuthorName'] = 'MADD AI'
    
    rmtree('generated_songs/' + song_id + '_g', ignore_errors=True)
    os.makedirs('generated_songs/' + song_id + '_g')

    copyfile('selected_songs/' + song_id + '/' + std_info['_coverImageFilename'], 
             'generated_songs/' + song_id + '_g/' + std_info['_coverImageFilename'])
    copyfile('selected_songs/' + song_id + '/' + std_info['_songFilename'], 
             'generated_songs/' + song_id + '_g/' + std_info['_songFilename'])
    
    with open('generated_songs/' + song_id + '_g/info.dat', 'w') as f:
        f.write(json.dumps(std_info))
    
    for i in range(len(levels)):
        
        with open('selected_songs/' + song_id + '/' + levels[i] + '.dat', 'r') as f:
            data = json.loads(f.read())
        
        times = sorted(list(set([el['_time'] for el in data['_notes']])))
        first_beats = ['n9n9n9n9010101n9n9n9n9n9']
        seq = evaluate(model, first_beats, len(times) - len(first_beats), temperature=temperature)

        notes = []

        for j in range(len(seq)):
            for k in range(0, 24, 2):
                el = seq[j][k:k+2]

                if el[0] != 'n':
                    notes.append({
                        '_time': times[j],
                        '_lineIndex': k // 2 % 4,
                        '_lineLayer': k // 2 // 4,
                        '_type': int(el[0]),
                        '_cutDirection': int(el[1])            
                    })
        
        data['_notes'] = notes
        
        with open('generated_songs/' + song_id + '_g/' + levels[i] + '.dat', 'w') as f:
            f.write(json.dumps(data))

In [16]:
for song_id in song_ids:
    for i in range(len(levels)):
        
        with open('selected_songs/' + song_id + '/' + levels[i] + '.dat', 'r') as f:
            data = json.loads(f.read())
        
        times = sorted(list(set([el['_time'] for el in data['_notes']])))
        first_beats = ['n9n9n9n9010101n9n9n9n9n9']
        seq = evaluate(model, first_beats, len(times) - len(first_beats), temperature=temperature)

        notes = []

        for j in range(len(seq)):
            for k in range(0, 24, 2):
                el = seq[j][k:k+2]

                if el[0] != 'n':
                    notes.append({
                        '_time': times[j],
                        '_lineIndex': k // 2 % 4,
                        '_lineLayer': k // 2 // 4,
                        '_type': int(el[0]),
                        '_cutDirection': int(el[1])            
                    })

In [34]:
print('[')
for el in notes:
    print('{')
    print(f""" "_time": {el['_time']}, "_lineIndex": {el['_lineIndex']}, "_lineLayer": {el['_lineLayer']}, "_type": {el['_type']}, "_cutDirection": {el['_cutDirection']} """)
    print('},')
print(']')

[
{
 "_time": 4, "_lineIndex": 0, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 4, "_lineIndex": 1, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 4, "_lineIndex": 2, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 4.5, "_lineIndex": 3, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 8, "_lineIndex": 0, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 8.5, "_lineIndex": 3, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 12, "_lineIndex": 0, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 12.5, "_lineIndex": 3, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 16, "_lineIndex": 0, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 16.5, "_lineIndex": 3, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 17, "_lineIndex": 0, "_lineLayer": 1, "_type": 0, "_cutDirection": 1 
},
{
 "_time": 17.5, "_lineIndex": 3, "_lineLayer": 1, "_type": 